
# Projekt: Regresja liniowa z regularyzacja L2, L1, L1+L2

## 1. Wczytanie danych i przygotowanie ich

In [17]:
import pandas as pd
df_train = pd.read_csv("/Users/../Data/dataset.train.txt", sep=" ", header=None)
df_naglowki = pd.read_csv("/Users/../Data/zmienne.txt", header=None)
df_test = pd.read_csv("/Users/..Data/dataset.test.txt", sep=" ", header=None)

In [18]:
# Usuniecie dwoch ostatnich pustych kolumn 
df_train = df_train.iloc[:, :-2]
df_test = df_test.iloc[:, :-2]

# Dodanie naglowkow do df
columns = df_naglowki[0].tolist()
df_train.columns = columns
df_test.columns = columns

# Polaczenie dwoch df
df_all = pd.concat([df_train, df_test], ignore_index=True)
df_all

,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33722,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974
33723,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771
33724,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051
33725,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699


## 2. Przygotowanie sie do sformułowania modelu regresji dla zmiennej objasniajacej 

In [19]:
df_all.isna().sum()
df_all.nunique()   
# zmienna 'unit_number' jest identyfikatorem - nie wchodzi do modelu bo jest bezuzyteczna 
# jest 8 zmiennych, które maja takie same wartosci
df_lm = df_all.drop(columns=[
    "unit_number", "op_setting_3", "sensor_measurement_1", "sensor_measurement_5", "sensor_measurement_6",
    "sensor_measurement_10", "sensor_measurement_16", "sensor_measurement_18", "sensor_measurement_19"
               ])

## 3. Budowa modelu regresji liniowej

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

y = df_lm["sensor_measurement_4"]
X = df_lm.drop(columns=["sensor_measurement_4"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

lm = LinearRegression()
lm.fit(X_train, y_train)

y_pred_train_linear = lm.predict(X_train)
y_pred_linear = lm.predict(X_test)

X = sm.add_constant(X)

lm = sm.OLS(y, X).fit()

# Opis modelu 
print(lm.summary())

                             OLS Regression Results                             
Dep. Variable:     sensor_measurement_4   R-squared:                       0.753
Model:                              OLS   Adj. R-squared:                  0.753
Method:                   Least Squares   F-statistic:                     6428.
Date:                  Sat, 25 Oct 2025   Prob (F-statistic):               0.00
Time:                          16:28:48   Log-Likelihood:                -96175.
No. Observations:                 33727   AIC:                         1.924e+05
Df Residuals:                     33710   BIC:                         1.925e+05
Df Model:                            16                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const     

R^2=0.754 Zmienne objasniajace wyjaśniaja w 75% zmienna objasniana, pozostale 25% jest nieznane. Jest to wysoka wartość.

Zmienne 'op_setting_1' op_setting_2' 'sensor_measurement_14' sa w modelu statystycznie nieistotne. (p-value> 0.05)

## 4. Budowa modelu regresji liniowej z dobraniem parametru alfa

In [21]:
# 1. Ridge (L2)
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)

y_pred_train_ridge = ridge.predict(X_train)
y_pred_ridge = ridge.predict(X_test)

print("R2 ridge test:", r2_score(y_test, y_pred_ridge))

R2 ridge test: 0.7489368352340551


In [22]:
# 2. Lasso (L1)
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1, max_iter=1000)
lasso.fit(X_train, y_train)

y_pred_train_lasso = lasso.predict(X_train)
y_pred_lasso = lasso.predict(X_test)

print("R2 lasso test:", r2_score(y_test, y_pred_lasso))

R2 lasso test: 0.7240711088618939


In [23]:
# 3. Elastic Net (L1 + L2)
from sklearn.linear_model import ElasticNet

elastic = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)
elastic.fit(X_train, y_train)

y_pred_train_elastic = elastic.predict(X_train)
y_pred_elasticnet = elastic.predict(X_test)

print("R2 ElasticNet:", r2_score(y_test, y_pred_elasticnet))

R2 ElasticNet: 0.7197711497867126


Ridge (L2): wybrałem alpha = 1.0, bo daje dobre dopasowanie R^2 na zbiorze testowym i zmniejsza wpływ silnie skorelowanych zmiennych.

Lasso (L1): wybrałem alpha = 0.1, ponieważ pozwala wyzerować nieistotne współczynniki, a R^2 nadal jest wysoki.

ElasticNet (L1+L2): alpha = 0.1, l1_ratio = 0.5 — kompromis między wygładzeniem współczynników (Ridge) a selekcją cech (Lasso).

## 5. Prezentacja współczynników wszystkich czterech modeli

In [29]:
print("R2 Linear:", r2_score(y_test, y_pred_linear))
print("R2 Ridge:", r2_score(y_test, y_pred_ridge))
print("R2 Lasso:", r2_score(y_test, y_pred_lasso))
print("R2 ElasticNet:", r2_score(y_test, y_pred_elasticnet))

R2 Linear: 0.7489353426812502
R2 Ridge: 0.7489368352340551
R2 Lasso: 0.7240711088618939
R2 ElasticNet: 0.7197711497867126


## 6. Ramka zawierajaca współczynniki determinacji dla wszystkich modeli treningowych i testowych.

In [31]:
r2_df = pd.DataFrame({
    "R2_train": [
        r2_score(y_train, y_pred_train_linear),
        r2_score(y_train, y_pred_train_ridge),
        r2_score(y_train, y_pred_train_lasso),
        r2_score(y_train, y_pred_train_elastic)
    ],
    "R2_test": [
        r2_score(y_test, y_pred_linear),
        r2_score(y_test, y_pred_ridge),
        r2_score(y_test, y_pred_lasso),
        r2_score(y_test, y_pred_elasticnet)
    ]
}, index=["Linear", "Ridge", "Lasso", "ElasticNet"])

print(r2_df)


            R2_train   R2_test
Linear      0.754086  0.748935
Ridge       0.754066  0.748937
Lasso       0.731897  0.724071
ElasticNet  0.727715  0.719771


## 7. Interpretacja różnic pomiedzy współczynnikami treningowymi i testowymi modelu.

Najwieksze R^2 daje liniowy model regresji.
Wszystkie modele są dobrze dopasowane – różnice między R^2 dla treningu i testu są minimalne, co oznacza dobrą generalizację.
Lasso i ElasticNet celowo wprowadzają mocniejszą regularizację, przez co lekko obniżają dopasowanie, ale zmniejszają ryzyko przeuczenia i poprawiają interpretowalność modelu.